Start by importing the packages we need.

In [1]:
import numpy as np
import kcorrect
import kcorrect.utils as ut
from astropy.cosmology import WMAP9 as cosmo

ModuleNotFoundError: No module named 'kcorrect'

Below I prepare load the file that contains the data into some numpy arrays

In [ ]:
print('Reading in LAMBDAR Catalog')

filein='/home/lrhunt/CATALOGS/PHOT/LAMBDAR_MAG_R.txt'
IDtot,RAtot,DECtot,utot,uerrtot,btot,berrtot,vtot,verrtot,rtot,rerrtot,itot,ierrtot,ztot,zerrtot,ktot,kerrtot,NUVtot,NUVerrtot,rh,zbesttot,zusetot,zphottot,SGCLASStot=np.loadtxt(filein,unpack=True)

The file is the full LAMBDAR catalog. I want to select galaxies with z<1.2 and spectroscopic redshifts only

In [ ]:
print('Selecting all galaxies between 0<z<1.2, Spectroscopic redshifts')

ID=IDtot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
RA=RAtot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
DECL=DECtot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
SGCLASS=SGCLASStot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
umag=utot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
uerr=uerrtot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
bmag=btot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
berr=berrtot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
vmag=vtot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
verr=verrtot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
imag=itot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
ierr=ierrtot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
zmag=ztot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
zerr=zerrtot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
kmag=ktot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
kerr=kerrtot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
zbest=zbesttot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
zuse=zusetot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
zphot=zphottot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
rmag=rtot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
rerr=rerrtot[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]
rh=rh[np.where((zbesttot>0)&(zbesttot<1.2)&(zusetot<=2)&(SGCLASStot==0))[0]]

k-correct uses a special unit which the label "maggies". We need to convert from apparent magnitude to maggies and from error in magnitude to inverse variance (invervar). Then put maggies into an array, and the error values into an array. (For more on how to calculate these values look at kcorrect.org or the python wrapper)

In [ ]:
print('Converting to maggies')

umaggies=ut.mag2maggies(umag)
bmaggies=ut.mag2maggies(bmag)
vmaggies=ut.mag2maggies(vmag)
rmaggies=ut.mag2maggies(rmag)
imaggies=ut.mag2maggies(imag)
zmaggies=ut.mag2maggies(zmag)

uinvervar=ut.invariance(umaggies,uerr)
binvervar=ut.invariance(bmaggies,berr)
vinvervar=ut.invariance(vmaggies,verr)
rinvervar=ut.invariance(rmaggies,rerr)
iinvervar=ut.invariance(imaggies,ierr)
zinvervar=ut.invariance(zmaggies,zerr)

allmaggies=np.stack((umaggies,bmaggies,vmaggies,rmaggies,imaggies,zmaggies),axis=-1)
allinvervar=np.stack((uinvervar,binvervar,vinvervar,rinvervar,iinvervar,zinvervar),axis=-1)

Next we need to make some arrays. First the coefficient array. This is where we store the coefficients the program uses to calculate maggies in the new passband. Then we need reconstructed maggies in the passband that we want to calculate. 

carr=coefficients array

rmarr=maggies estimated based on the coefficients (basically we can calculate the magnitude of the source based on the spectrum (or SED) that kcorrect fits to the soruce). This can be used to see how the observed and estimated magnitudes compare

rmarr0=The rest frame maggies for the object. Used to calculate the rest-frame magnitude of the source in the original filter

rmarr0B=Rest from maggies in the Johnson B Band. 

rmarr0V=Rest from maggies in the Johnson V Band.

rmarr0U=Rest from maggies in the Johnson U Band.

In [ ]:
carr=np.ndarray((len(bmaggies),6))
rmarr=np.ndarray((len(bmaggies),7))
rmarr0=np.ndarray((len(bmaggies),7))
rmarr0B=np.ndarray((len(bmaggies),7))
rmarr0V=np.ndarray((len(bmaggies),7))
rmarr0U=np.ndarray((len(bmaggies),7))

Finally we calculate the k-correction. First we need to load the filter template we need

In [ ]:
print('Computing k-corrections and estimated magnitudes')

kcorrect.load_templates()
kcorrect.load_filters('/home/lrhunt/programs/kcorrect/data/templates/Lum_Func_Filters_US.dat')

Example of the template file. You can create this (or similar) and store in the templates directory. Example of the path I used can be seen above. 

KCORRECT_DIR
data/filters/capak_cfht_megaprime_sagem_u.par
data/filters/capak_subaru_suprimecam_B.par
data/filters/capak_subaru_suprimecam_V.par
data/filters/capak_subaru_suprimecam_r.par
data/filters/capak_subaru_suprimecam_i.par
data/filters/capak_subaru_suprimecam_z.par

Next we calculate the coefficients and put them in the coefficients array. 

In [ ]:
for i in range(0,len(carr)):
	carr[i]=kcorrect.fit_nonneg(zbest[i],allmaggies[i],allinvervar[i])

Next we use the coefficients to reconstruct the maggies (i.e. use the SED to calculate the maggies in the observed passband and the emitted passband)

In [ ]:
for i in range(0,len(carr)):
	rmarr[i]=kcorrect.reconstruct_maggies(carr[i])
	rmarr0[i]=kcorrect.reconstruct_maggies(carr[i],redshift=0)

Now we need to load the template which only contains the b-band filter. This will allow us to estimate maggies for Johnson B.

In [ ]:
kcorrect.load_templates()
kcorrect.load_filters('/home/lrhunt/programs/kcorrect/data/templates/BESSEL_B2.dat')

for i in range(0,len(carr)):
	rmarr0B[i]=kcorrect.reconstruct_maggies(carr[i],redshift=0)

Next we calculate the k-correction. 

kcorr=correct from observed to rest frame magnitude in the the observed filters

kcorrM=k-correction from the observed apparent magnitude to the restframe johnson B magnitude

corrB=apparent Johnson B magnitude calculated from the estimated SED

In [ ]:
kcorr=-2.5*np.log10(rmarr/rmarr0)
kcorrM=-2.5*np.log10(rmarr/rmarr0B)
corrB=-2.5*np.log10(rmarr0B)+0.09

Finally calculate the absolute magnitude. 

I used the apparent magnitude in the passband closest to the rest frame Johnson B band. This reduces the required k-correction and improves the estimate! The small value (i.e. in the first loop -0.05122) corrects for the difference in observed apparent magnitude (from survey) and calculated apparent magnitude (what kcorrect estimates the apparent magnitude to be based on the constructed SED). This is similar to what is done in CAPAK,2007 (the original COSMOS photometry paper)

In [ ]:
M=np.zeros_like(zbest)
bv=corrB[:,3]-corrV[:,4]
#M=corrB[:,3]-cosmo.distmod(zbest).value
for i in range(0,len(zbest)):
	if zbest[i]<=0.1:
		M[i]=bmag[i]-0.05122-cosmo.distmod(zbest[i]).value-kcorrM[i][2]
	if zbest[i]<=0.35 and zbest[i]>0.1:
		M[i]=vmag[i]+0.069802-cosmo.distmod(zbest[i]).value-kcorrM[i][3]
	if zbest[i]<=0.55 and zbest[i]>0.35:
		M[i]=rmag[i]-0.01267-cosmo.distmod(zbest[i]).value-kcorrM[i][4]
	if zbest[i]<=0.75 and zbest[i]>0.55:
		M[i]=imag[i]-0.004512-cosmo.distmod(zbest[i]).value-kcorrM[i][5]
	if zbest[i]>0.75:
		M[i]=zmag[i]-0.00177-cosmo.distmod(zbest[i]).value-kcorrM[i][6]

M=M+0.09

If you have any questions about any of this just ask! Hopefully what I've supplied is helpful and makes sense!